In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import copy

import yfinance as yf

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

In [2]:
stocks = pd.read_csv('stocks_by_sector.csv')
stocks

,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology
0,T,F,KO,MRO,BAC,PFE,GE,AAPL
1,TWTR,GM,KR,KMI,WFC,MRK,DAL,AMD
2,FB,EBAY,PG,XOM,C,GILD,CSX,MU
3,CMCSA,SBUX,WMT,HAL,RF,BMY,AAL,MSFT
4,VZ,NKE,MDLZ,WMB,JPM,BSX,LUV,INTC
5,NFLX,M,MO,COP,KEY,ABT,FAST,CSCO
6,DIS,MGM,COTY,SLB,MS,CVS,CAT,HPE
7,ATVI,TJX,WBA,DVN,HBAN,ABBV,JCI,ORCL
8,IPG,TGT,PM,CVX,SCHW,JNJ,UAL,NVDA
9,DISCA,NWL,PEP,COG,SYF,MDT,UNP,AMAT


------------------------------------
**Get the data and the indicators**
------------------------------------
We collect the stock data from yfinance for between dates `'2016-01-02'-'2019-01-01'` (3 years) and calculate certain indicators helpful for analysis, which are:
- Relative Strength Index (RSI)
- Stochastic Oscillator
- Williams %R
- Moving Average Convergence Divergnece (MACD)
- Price Rate Of Change
- On Balance Volume

After calculating these indicators for each day, we add them to the dataframe of the stock as new columns. We select the columns `['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume']` at the end to obtain a new dataframe representing the stock.

We do this for each of the stocks we have selected and concatenate the individual dataframes to obtain a a resulting dataframe of shape `(T, NxM)`, where `T` is the number of time steps, `N` the number of assets, `M` the number of indicators for each asset.

We also standardize the names of individual assets (e.g. `asset_1`) for tensorization.

Also Note that the labels are set according to the price change $n$ days-out (set to be 9 below).

In [3]:
from GRTEL.indicators import get_RSI, get_Stochastic_Oscillator, get_Williams, get_MACD, get_OBV

In [4]:
samples = []
labels = []
for sector in stocks.columns:
    for i, stock in enumerate(stocks[sector].values):
        
        stock_name = 'asset_' + str(i+1)
        
        # get the original data
        data = yf.download(stock, start='2016-01-02', end='2019-01-01')

        # calculate change in price
        data['change_in_price'] = data['Close'].diff()

        # calculate indicators
        data['RSI'] = get_RSI(data)
        data['k_percent'] = get_Stochastic_Oscillator(data)
        data['r_percent'] = get_Williams(data)

        # Calculate the MACD
        macd, ema_9_macd = get_MACD(data)
        data['MACD'] = macd
        data['MACD_EMA9'] = ema_9_macd

        # Calculate the 9-day Price Rate of Change
        data['Price Rate Of Change'] = data['Close'].pct_change(periods=9)

        # Calculate On Balance Volume
        data['On Balance Volume'] = get_OBV(data)

        # Create the predicition column (To keep this as a binary classifier we'll consider flat days as up days)
        days_out = 9
        data['Prediction'] = np.sign(np.sign(data['Close'].shift(-days_out) - data['Close']) + 1.)

        # Drop rows with NaN.
        data = data.dropna()

        X_i = data[['Close', 'RSI', 'k_percent', 'r_percent', 'MACD', 'MACD_EMA9', 'Price Rate Of Change', 'On Balance Volume']].copy()
        X_i.columns = [[sector]*len(X_i.columns), [stock_name]*len(X_i.columns), X_i.columns]
        
        y_i = data['Prediction'].copy()
        y_i.name = stock
        
        samples.append(X_i)
        labels.append(y_i)

        
samples = pd.concat(samples, axis=1)
samples.columns.names = ['Sector', 'Asset', 'Metrics']

labels = pd.concat(labels, axis=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [5]:
samples

Sector     Communication Services                                             \
Asset                     asset_1                                              
Metrics                     Close        RSI  k_percent  r_percent      MACD   
Date                                                                           
2016-01-22              35.139999  66.984998  97.740062  -2.259938  0.092199   
2016-01-25              35.000000  63.461411  78.712861 -21.287139  0.123844   
2016-01-26              35.400002  68.861369  92.129714  -7.870286  0.170558   
2016-01-27              35.480000  69.888295  86.610888 -13.389112  0.209019   
2016-01-28              35.529999  70.587814  88.702907 -11.297093  0.239060   
...                           ...        ...        ...        ...       ...   
2018-12-11              29.700001  36.639813  18.662008 -81.337992 -0.234568   
2018-12-12              30.160000  46.469108  34.859168 -65.140832 -0.231801   
2018-12-13              29.910000  42.349247  26.056346 -73.943654 -0.246935   
2018-12-14              30.219999  48.838972  36.971827 -63.028173 -0.231248   
2018-12-17              29.860001  42.437767  24.295808 -75.704192 -0.245041   

Sector                                                                  \
Asset                                                          asset_2   
Metrics    MACD_EMA9 Price Rate Of Change On Balance Volume      Close   
Date                                                                     
2016-01-22  0.021266             0.047704          10592300  17.840000   
2016-01-25  0.042529             0.030928         -19672100  17.020000   
2016-01-26  0.068877             0.044248          22005000  17.010000   
2016-01-27  0.097551             0.051571          72268000  16.780001   
2016-01-28  0.126372             0.035860          98452400  16.490000   
...              ...                  ...               ...        ...   
2018-12-11 -0.238566            -0.024310          26518900  34.450001   
2018-12-12 -0.237213            -0.018868          67422600  36.250000   
2018-12-13 -0.239157            -0.021590          18324800  35.889999   
2018-12-14 -0.237575            -0.032650          61066200  35.869999   
2018-12-17 -0.239068            -0.058341          17044500  33.430000   

Sector                 ... Information Technology                    \
Asset                  ...                asset_9                     
Metrics           RSI  ...   Price Rate Of Change On Balance Volume   
Date                   ...                                            
2016-01-22  30.079617  ...              -0.039824          -9242700   
2016-01-25  23.781919  ...              -0.042453         -16010900   
2016-01-26  23.712060  ...              -0.049039         -10312300   
2016-01-27  21.997274  ...              -0.030759         -16104800   
2016-01-28  19.903236  ...              -0.021625         -23024200   
...               ...  ...                    ...               ...   
2018-12-11  65.458987  ...              -0.036037        1013352200   
2018-12-12  74.545905  ...              -0.069782        1029705600   
2018-12-13  70.279251  ...              -0.053826        1017921000   
2018-12-14  70.022338  ...              -0.103898        1006125500   
2018-12-17  46.232094  ...              -0.155610         989553800   

Sector                                                                      \
Asset        asset_10                                                        
Metrics         Close        RSI  k_percent  r_percent      MACD MACD_EMA9   
Date                                                                         
2016-01-22  17.049999  51.119297  43.165445 -56.834555 -0.097708 -0.124555   
2016-01-25  16.910000  47.801520  38.129494 -61.870506 -0.078060 -0.114917   
2016-01-26  17.350000  57.746426  63.025232 -36.974768 -0.035484 -0.098570   
2016-01-27  17.090000  51.107526  70.454523 -29.545477 -0.018663 -0.082221  

In [6]:
labels

,T,TWTR,FB,CMCSA,VZ,NFLX,DIS,ATVI,IPG,DISCA,...,AAPL,AMD,MU,MSFT,INTC,CSCO,HPE,ORCL,NVDA,AMAT
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-22,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
2016-01-25,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-26,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-27,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2016-01-28,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-12-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


-------
Store samples and labels as .csv files
-----

In [7]:
# samples.to_csv('samples_sector.csv')
# labels.to_csv('labels_sector.csv')